In [1]:
import sys
sys.path.insert(0, '../nsga-net/')
from search import train_search
import os
import pandas as pd
import plotly.graph_objs as go
import hashlib
import plotly.io as pio
import ast
nas_log_dir = 'logs/search-GA-BiObj-micro-20240416-173204'

/home/nima/miniconda3/envs/bittensor/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nima/miniconda3/envs/bittensor/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
def draw_pareto_graph(log_directory):
    accuracy_values = []
    num_parameters_values = []
    file_names = []
    assigned_users = []

    for filename in os.listdir(log_directory):
        if filename.endswith('.csv'):
            filepath = os.path.join(log_directory, filename)
            df = pd.read_csv(filepath, skiprows=lambda x: x % 3 != 0)
            if not df.empty:
                for index, row in df.iterrows():
                    response_list = eval(row['Response'])
                    accuracy_values.append(response_list[0])
                    num_parameters_values.append(response_list[1])
                    file_names.append(filename)
                    assigned_users.append(row.get('Assigned_User', 'N/A'))

    # Check if the lists are empty
    if not num_parameters_values or not accuracy_values:
        # Create an empty plot with a consistent style
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=[], y=[], mode='markers'))
        fig.update_layout(
            title='No data available',
            xaxis=dict(
                title='Number of Parameters',
                title_font=dict(color='#FFFFFF'),
                tickfont=dict(color='#FFFFFF'),
                linecolor='#FFFFFF',
                showgrid=False,
                range=[0, 1]
            ),
            yaxis=dict(
                title='Accuracy',
                title_font=dict(color='#FFFFFF'),
                tickfont=dict(color='#FFFFFF'),
                linecolor='#FFFFFF',
                showgrid=False,
                range=[0, 1]
            ),
            plot_bgcolor='#171B23',
            paper_bgcolor='#171B23',
            legend=dict(font=dict(color='#FFFFFF')),
            hovermode='closest',
            hoverlabel=dict(font=dict(color='#FFFFFF')),
            showlegend=False
        )
        return pio.to_html(fig, full_html=False)

    # Compute unique colors for each file
    unique_colors = [hashlib.sha256(name.encode()).hexdigest()[:6] for name in set(file_names)]

    scatter_traces = []
    for filename, color in zip(set(file_names), unique_colors):
        x_values = [num_parameters_values[i] for i, name in enumerate(file_names) if name == filename]
        y_values = [accuracy_values[i] for i, name in enumerate(file_names) if name == filename]
        hover_text = [f"uid:{assigned_users[i]}" for i, name in enumerate(file_names) if name == filename]
        scatter_trace = go.Scatter(
            x=x_values,
            y=y_values,
            mode='markers',
            name=filename.split('.')[0],
            marker=dict(color=f'#{color}'),
            hoverinfo='text',
            text=hover_text
        )
        scatter_traces.append(scatter_trace)

    pareto_optimal_points = find_pareto_optimal_points(accuracy_values, num_parameters_values)
    print(pareto_optimal_points)
    padding_factor = 0.01  # 10% padding
    x_min, x_max = min(num_parameters_values), max(num_parameters_values)
    y_min, y_max = min(accuracy_values), max(accuracy_values)
    x_range = [(1 - padding_factor) * x_min, (1 + padding_factor) * x_max]
    y_range = [(1 - padding_factor) * y_min, (1 + padding_factor) * y_max]

    fig = go.Figure(data=scatter_traces)
    if pareto_optimal_points:
        pareto_x, pareto_y = zip(*pareto_optimal_points)
        pareto_trace = go.Scatter(
            x=pareto_x,
            y=pareto_y,
            mode='lines',
            name='Pareto Optimal Line',
            line=dict(color='#ff0000')
        )
        fig.add_trace(pareto_trace)

    fig.update_layout(
        title='Scatter Plot of Multi-Objective Optimization Scores',
        xaxis=dict(
            title='Number of Parameters',
            range=x_range,
            title_font=dict(color='#FFFFFF'),
            tickfont=dict(color='#FFFFFF'),
            linecolor='#FFFFFF',
            showgrid=False
        ),
        yaxis=dict(
            title='Accuracy',
            range=y_range,
            title_font=dict(color='#FFFFFF'),
            tickfont=dict(color='#FFFFFF'),
            linecolor='#FFFFFF',
            showgrid=False
        ),
        plot_bgcolor='#171B23',
        paper_bgcolor='#171B23',
        legend=dict(font=dict(color='#FFFFFF')),
        hovermode='closest',
        hoverlabel=dict(font=dict(color='#FFFFFF')),
        showlegend=False
    )

    return fig

def find_pareto_optimal_points(accuracy, parameters):
    pareto_optimal = []
    for i in range(len(accuracy)):
        is_pareto = True
        for j in range(len(accuracy)):
            if accuracy[i] <= accuracy[j] and parameters[i] >= parameters[j] and i != j:
                is_pareto = False
                break
        if is_pareto:
            pareto_optimal.append((parameters[i], accuracy[i]))
    pareto_optimal.sort()
    return pareto_optimal

In [4]:
import pandas as pd
import os
import hashlib
import plotly.graph_objs as go
import plotly.io as pio

def print_pareto(log_directory):
    accuracy_values = []
    num_parameters_values = []
    file_names = []
    assigned_users = []
    genome_values = []  # Store Genome values

    for filename in os.listdir(log_directory):
        if filename.endswith('.csv'):
            filepath = os.path.join(log_directory, filename)
            df = pd.read_csv(filepath, skiprows=lambda x: x % 3 != 0)
            for index, row in df.iterrows():
                response_list = eval(row['Response'])
                accuracy_values.append(response_list[0])
                num_parameters_values.append(response_list[1])
                file_names.append(filename)
                assigned_users.append(row.get('Assigned_User', 'N/A'))
                genome_values.append(row['Genome'])  # Collect Genome values

    # Identify Pareto optimal points
    pareto_indices = find_pareto(accuracy_values, num_parameters_values)

    return pareto_indices, genome_values, assigned_users

def find_pareto(accuracy, parameters):
    pareto_optimal_indices = []
    for i in range(len(accuracy)):
        is_pareto = True
        for j in range(len(accuracy)):
            if accuracy[i] <= accuracy[j] and parameters[i] >= parameters[j] and i != j:
                is_pareto = False
                break
        if is_pareto:
            pareto_optimal_indices.append(i)
    return pareto_optimal_indices

pareto_indices, genome_values, assigned_users = print_pareto(nas_log_dir)
# This is selcted from the Pareto Optimal Line from the figure below
which_genome = 156
for i in range(len(pareto_indices)):
    if assigned_users[pareto_indices[i]] == which_genome:
            genome_string_str = genome_values[pareto_indices[i]]
            genome_string = ast.literal_eval(genome_string_str)
            print(genome_string)
    



[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]


In [6]:
fig = draw_pareto_graph(nas_log_dir)
fig.show()

[(0.027478, 72.72), (0.036622, 78.2), (0.048358, 79.55), (0.05095, 80.47), (0.063982, 80.86), (0.065278, 81.54), (0.066574, 81.61), (0.073054, 82.37), (0.075718, 82.99), (0.087454, 83.87), (0.092638, 84.44), (0.097822, 84.45), (0.134542, 86.78), (0.158014, 86.92), (0.15931, 87.2), (0.171046, 87.6), (0.181342, 88.08), (0.193078, 88.17), (0.20611, 88.47), (0.220438, 88.49), (0.221734, 88.51), (0.22303, 88.71), (0.232174, 88.79), (0.239878, 88.99), (0.267238, 89.03), (0.26983, 89.09), (0.278974, 89.22), (0.284086, 89.24), (0.285454, 89.26), (0.292006, 89.56), (0.299782, 89.6), (0.319222, 89.62), (0.329662, 89.88), (0.392086, 89.95), (0.393382, 89.96), (0.456958, 89.97), (0.459622, 90.01), (0.460846, 90.02), (0.462214, 90.17), (0.46999, 90.32), (0.486766, 90.5), (0.599014, 90.72), (0.613342, 90.76), (0.62119, 90.8), (0.626374, 90.86), (0.67447, 90.99), (0.738118, 91.06), (0.743302, 91.14), (0.788806, 91.2), (0.797878, 91.37), (0.86527, 91.39), (0.874342, 91.46), (0.905518, 91.5), (0.921142

In [51]:
performance = train_search.main(genome=genome_string,
                                        search_space = 'macro',
                                        init_channels = 36,
                                        layers=15, cutout=True,
                                        epochs=400,
                                        save='arch_{}'.format(1),
                                        expr_root='')

Experiment dir : arch_1
04/19 01:28:03 AM Architecture = [[[1], [0, 0], [0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 0, 1], [0]], [[0], [0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1]], [[0], [0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1]]]
04/19 01:28:03 AM param size = 0.066574MB
2024-04-19 01:28:04.548 |       INFO       | 👷 epoch 0 lr 0.024999228947065064


/home/nima/miniconda3/envs/bittensor/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:143: UserWarning:

Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate

/home/nima/miniconda3/envs/bittensor/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:854: UserWarning:

To get the last learning rate computed by the scheduler, please use `get_last_lr()`.



2024-04-19 01:28:09.977 |       INFO       | 👷 train_acc 32.9525           
2024-04-19 01:28:10.643 |       INFO       | 👷 valid_acc 42.41             
2024-04-19 01:28:10.644 |       INFO       | 👷 epoch 1 lr 0.02499730139498813
2024-04-19 01:28:16.292 |       INFO       | 👷 train_acc 45.255            
2024-04-19 01:28:16.979 |       INFO       | 👷 valid_acc 41.59             
2024-04-19 01:28:16.979 |       INFO       | 👷 epoch 2 lr 0.024994602998053037
2024-04-19 01:28:22.551 |       INFO       | 👷 train_acc 50.305            
2024-04-19 01:28:23.174 |       INFO       | 👷 valid_acc 54.53             
2024-04-19 01:28:23.174 |       INFO       | 👷 epoch 3 lr 0.024991133922707416
2024-04-19 01:28:28.697 |       INFO       | 👷 train_acc 53.2225           
2024-04-19 01:28:29.340 |       INFO       | 👷 valid_acc 58.58             
2024-04-19 01:28:29.341 |       INFO       | 👷 epoch 4 lr 0.024986894382937972
2024-04-19 01:28:34.736 |       INFO       | 👷 train_acc 55.9675           
2